# Preferencias de Comercios de Clientes de Tarjeta de Crédito

El proposito de esta iniciativa es poder determinar cuales son los tipos de comercios preferidos por nuestros clientes, crear un algoritmo que a partir de las transacciones de un cliente determine cuales son sus categorias de comercios preferidos, poder medir que tan preferente es este comercio para poder ordenarlos, y usar los top n comercios preferidos. 

Esta presentación esta dividida en secciones. Puedes hacer clic en las flechas para navegar, o utilizar la tecla SPACE para avanzar según el orden de la presentación.

In [1]:
import pandas as pd 
import numpy as np
import qgrid
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot as ipy
import  plotly
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

plotly.offline.init_notebook_mode(connected=True)
plotly_config={'showLink': False}

# Un vistazo a los datos

Para esta proyecto, fueron utilizados las transacciones de Tarjetas de Crédito de clientes de BSC en el año 2018, resumidas por comercio, por mes, por cliente. Este resumen contiene el valor total de las transacciones y la cantidad total de transacciones realizadas por el cliente en un comercio particular.

In [2]:
transacciones =  pd.read_csv("data/Transacciones_TC_Pre_Categoria_Sample.zip",compression='zip')
                                    
transacciones.head()

,Codigo_Cliente,DESCRI_PRODUCTO,Moneda,Anio,Mes,Cantidad,Valor_Total_RD,Comercios_Replace,Categoria_Nueva,CODIGO_CATEGORIA_NUEVO
0,230582,VISA CLASICA,USD,2018,5,1,3999.5,Abaenglish,Colegios Y Servicios Educaci,8299
1,326182,VISA ORO,USD,2018,3,1,3999.5,Abaenglish,Colegios Y Servicios Educaci,8299
2,126595,VISA CLASICA,USD,2018,9,1,749.5,Abaenglish,Colegios Y Servicios Educaci,8299
3,350836,VISA CLASICA,USD,2018,6,2,3000.0,Crehana,Colegios Y Servicios Educaci,8299
4,182727,VISA ORO,USD,2018,11,1,808.5,Crehana,Colegios Y Servicios Educaci,8299


# Preparación de Categorias

Lo primero que hicimos fue consolidar las categorias de comercios de transacciones de Tarjetas. Esto es debido a que existen más de 300 categorias, las cuales pueden ser agrupadas por sus similitudes.

Vamos primero a visualizar las categorias y sus comercios utilizando una tabla interactiva. Puede filtrar y buscar en esta tabla para explorar los datos.

In [3]:
transacciones_productos_categoria = (transacciones.groupby(["CODIGO_CATEGORIA_NUEVO","Categoria_Nueva","Comercios_Replace"])
                                                  [['Cantidad','Valor_Total_RD']]
                                                  .sum()
                                                  .reset_index()
                                                  .sort_values(['Cantidad','Valor_Total_RD'],ascending=[False,False]))
productos_categoria_widget = qgrid.show_grid(transacciones_productos_categoria)

In [4]:
productos_categoria_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Se pueden observar las categorias que facturan más tanto en cantidad como en montos, y sus comercios más populares. Sin embargo, analizar este inmenso numero de categorias no es muy sencillo. Afortunadamente, las categorias pueden ser agrupadas, por ejemplo, todas las relacionadas a lineas aereas y viajes en una sola categoria. Para ayudarnos, los primeros digitos de un codigo de categoria usualmente representan una categoria en comun. 

Vamos a explorar estos codigos para crear una tabla de reemplazos. 

In [5]:
categorias = transacciones[['CODIGO_CATEGORIA_NUEVO','Categoria_Nueva']].drop_duplicates()
categorias = categorias.sort_values(by='CODIGO_CATEGORIA_NUEVO')
categorias.CODIGO_CATEGORIA_NUEVO = categorias.CODIGO_CATEGORIA_NUEVO.astype(str)
categorias.head()

,CODIGO_CATEGORIA_NUEVO,Categoria_Nueva
407662,0,Ath
1827461,742,Veterinarias
1911814,763,Ventas Productos Agricolas
1970698,780,Ladscaping And Horticultural Servises
1970583,1520,General Contractors Resid And Comercial


In [6]:
categorias['CODIGO_CATEGORIA_NUEVO'] = categorias['CODIGO_CATEGORIA_NUEVO'].str.pad(4,'left','0')
categorias['head_codigo'] = categorias.CODIGO_CATEGORIA_NUEVO.apply(lambda x: x[:2])
categorias['index_codigo'] = categorias.CODIGO_CATEGORIA_NUEVO.apply(lambda x: x[0])
qgrid_categorias = qgrid.show_grid(categorias)
categorias.head()

,CODIGO_CATEGORIA_NUEVO,Categoria_Nueva,head_codigo,index_codigo
407662,0000,Ath,00,0
1827461,0742,Veterinarias,07,0
1911814,0763,Ventas Productos Agricolas,07,0
1970698,0780,Ladscaping And Horticultural Servises,07,0
1970583,1520,General Contractors Resid And Comercial,15,1


In [7]:
qgrid_categorias

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Segun estos codigos, creamos una tabla para agrupar las categorias de comercios segun los dos primeros digitos de la categoria.

In [8]:
head_codigo_replacement_dict = {
    '07': 'Servicios Agropecuarios',
    '00': 'Ath',
    '15': 'Servicios de Construccion y Decoracion',
    '17': 'Servicios de Construccion y Decoracion',
    '27': 'Servicios de Textiles e Imprenta',
    '28': 'Servicios de Textiles e Imprenta',
    '30': 'Servicios de Transporte y Viajes',
    '31': 'Servicios de Transporte y Viajes',
    '32': 'Servicios de Transporte y Viajes',
    '33': 'Alquiler de Automoviles',
    '34': 'Alquiler de Automoviles',
    '35': 'Hoteles Moteles Resorts',
    '36': 'Hoteles Moteles Resorts',
    '37': 'Hoteles Moteles Resorts',
    '40': 'Servicios de Transporte y Viajes',
    '41': 'Servicios de Transporte y Viajes',
    '42': 'Servicios de Transporte y Viajes',
    '44': 'Servicios de Transporte y Viajes',
    '45': 'Servicios de Transporte y Viajes',
    '47': 'Servicios de Transporte y Viajes',
    '48': 'Servicios de Comunicacion, Internet y Telecable',
    '49': 'Ventas y Servicios de Equipos Electricos',
    '50': 'Ventas y Servicios de Equipos Electricos',
    '51': 'Articulos de Oficina',
    '52': 'Ferreterias y Hogar',
    '53':  'Ropas',
    '54': 'Supermercados y Alimentos',
    '55': 'Gasolineras y Vehiculos',
    '56': 'Ropas',
    '57': 'Ferreterias y Hogar',
    '59': 'Tienda De Venta Especializada',
    '60': 'Instituciones Financieras y Aseguradoras',
    '62': 'Instituciones Financieras y Aseguradoras',
    '63': 'Instituciones Financieras y Aseguradoras',
    '65': 'Instituciones Financieras y Aseguradoras',
    '70': 'Hoteles Moteles Resorts',
    '72': 'Servicios de Salud, Belleza y Recreacion',
    '73': 'Otros Servicios',
    '75' : 'Gasolineras y Vehiculos',
    '76' : 'Ventas y Servicios de Equipos Electricos',
    '78': 'Cine y Peliculas',
    '79': 'Servicios de Salud, Belleza y Recreacion',
    '80': 'Servicios de Salud, Belleza y Recreacion',
    '81': 'Otros Servicios',
     '82': 'Servicios Educativos',
    '83': 'Otros Servicios',
    '86': 'Otros Servicios',
    '87' : 'Servicios de Salud, Belleza y Recreacion',
    '89': 'Otros Servicios',
    '92': 'Servicios Gubernamentales',
    '93': 'Servicios Gubernamentales',
    '94': 'Servicios Gubernamentales'
}

codigo_replacement_dict = {
    '5814':'Comida Rapida',
    '5812': 'Restaurantes',
    '5811': 'Licores',
    '5813': 'Vida Nocturna',
    '5734': 'Tecnologia y Software',
    '5732': 'Musica',
    '5733': 'Musica',
    '5735' : 'Musica',
    '5921':  'Licores',
    '7299' : 'Servicio Personales (Deliveries, etc.)',
    '5912': 'Servicios de Salud, Belleza y Recreacion'
}

head_codigo_df = pd.DataFrame({"head_codigo":list(head_codigo_replacement_dict.keys()),
                               "Agrupacion": list(head_codigo_replacement_dict.values())})
head_codigo_df_merge = head_codigo_df.merge(categorias)
for codigo,repl in codigo_replacement_dict.items():
    head_codigo_df_merge.loc[head_codigo_df_merge.CODIGO_CATEGORIA_NUEVO == codigo, 'Agrupacion'] = repl
head_codigo_df_qgrid = qgrid.show_grid(head_codigo_df_merge)

In [9]:
head_codigo_df_qgrid 

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [10]:
categorias['Categoria_Agrupada'] = categorias.head_codigo.replace(head_codigo_replacement_dict)
for codigo,repl in codigo_replacement_dict.items():
    categorias.loc[categorias.CODIGO_CATEGORIA_NUEVO == codigo, 'Categoria_Agrupada'] = repl
categorias.head()

,CODIGO_CATEGORIA_NUEVO,Categoria_Nueva,head_codigo,index_codigo,Categoria_Agrupada
407662,0000,Ath,00,0,Ath
1827461,0742,Veterinarias,07,0,Servicios Agropecuarios
1911814,0763,Ventas Productos Agricolas,07,0,Servicios Agropecuarios
1970698,0780,Ladscaping And Horticultural Servises,07,0,Servicios Agropecuarios
1970583,1520,General Contractors Resid And Comercial,15,1,Servicios de Construccion y Decoracion


In [11]:
categorias['CODIGO_CATEGORIA_NUEVO'] = categorias['CODIGO_CATEGORIA_NUEVO'].astype(int)
categorias.head()

,CODIGO_CATEGORIA_NUEVO,Categoria_Nueva,head_codigo,index_codigo,Categoria_Agrupada
407662,0,Ath,00,0,Ath
1827461,742,Veterinarias,07,0,Servicios Agropecuarios
1911814,763,Ventas Productos Agricolas,07,0,Servicios Agropecuarios
1970698,780,Ladscaping And Horticultural Servises,07,0,Servicios Agropecuarios
1970583,1520,General Contractors Resid And Comercial,15,1,Servicios de Construccion y Decoracion


In [12]:
transacciones = transacciones.merge(categorias[['CODIGO_CATEGORIA_NUEVO','Categoria_Agrupada']])
transacciones = transacciones[['Codigo_Cliente','DESCRI_PRODUCTO','Anio','Mes','Cantidad','Valor_Total_RD',
                              'Comercios_Replace','Categoria_Nueva','Categoria_Agrupada']]

Con esto, agregamos una nueva agrupación a las transacciones, con la cual haremos el análisis.

In [13]:
transacciones.head()

,Codigo_Cliente,DESCRI_PRODUCTO,Anio,Mes,Cantidad,Valor_Total_RD,Comercios_Replace,Categoria_Nueva,Categoria_Agrupada
0,230582,VISA CLASICA,2018,5,1,3999.5,Abaenglish,Colegios Y Servicios Educaci,Servicios Educativos
1,326182,VISA ORO,2018,3,1,3999.5,Abaenglish,Colegios Y Servicios Educaci,Servicios Educativos
2,126595,VISA CLASICA,2018,9,1,749.5,Abaenglish,Colegios Y Servicios Educaci,Servicios Educativos
3,350836,VISA CLASICA,2018,6,2,3000.0,Crehana,Colegios Y Servicios Educaci,Servicios Educativos
4,182727,VISA ORO,2018,11,1,808.5,Crehana,Colegios Y Servicios Educaci,Servicios Educativos


In [14]:
transacciones = transacciones.loc[transacciones.Categoria_Agrupada != "Ath"]
transacciones.Categoria_Agrupada = transacciones.Categoria_Agrupada.str.replace("Alquier|Alquiter|Alquilter","Alquiler")

# Preparación de Datos

#### Fecha

Ya que las transacciones no incluyen la fecha, sino que estan resumidas por mes, agruparemos todas las transacciones al primer dia de cada mes, y crearemos una fecha con ellas.

In [15]:
transacciones["Fecha"] = pd.to_datetime(transacciones.Anio.astype(str) + "-"\
                         + transacciones.Mes.astype(str).str.pad(2,"left","0")\
                         + "-01")
transacciones.Fecha.head()

0   2018-05-01
1   2018-03-01
2   2018-09-01
3   2018-06-01
4   2018-11-01
Name: Fecha, dtype: datetime64[ns]

#### Meses Anteriores 

Para determinar la recencia de las transacciones, calculamos los meses anteriores a la fecha maxima de los datos, en este caso Diciembre 2018.

In [16]:
def month_diff_indiv(a, b):
    return 12 * (a.year - b.dt.year) + (a.month - b.dt.month)
transacciones["Meses_Anteriores"] = month_diff_indiv(transacciones.Fecha.max(), transacciones.Fecha)

In [17]:
transacciones[['Fecha','Meses_Anteriores']].head()

,Fecha,Meses_Anteriores
0,2018-05-01,7
1,2018-03-01,9
2,2018-09-01,3
3,2018-06-01,6
4,2018-11-01,1


In [18]:
transacciones = transacciones.loc[transacciones.Meses_Anteriores <= 12]
transacciones.shape

(1981931, 11)

Por último, establecemos la configuración de las variables a analizar. Estas son:

- Agrupaciones de Interes: Cliente, Producto, Categoria Agrupada de Comercio
- Identificador: Cliente + Producto
- Valores a Medir: Cantidad, Monto Total Transaccionado, Recencia de Transacciones.
- Top N Comercios: Top 3 Comercios segun condiciones.

In [19]:
# CONFIG VARS
# Nivel de Interes: Cliente. Puede ser cliente y producto. 
categoria_comercio_interes = ['DESCRI_PRODUCTO','Categoria_Agrupada']
interes_group = ['Codigo_Cliente'] + categoria_comercio_interes
identificador_group = ['Codigo_Cliente','DESCRI_PRODUCTO']
valores_medida = ['Cantidad','Valor_Total_RD']
top_n = 3

# Metodos para Calcular Preferencias de Comercios

## Por Cantidad: Mientras más transacciones mejor

In [20]:
transacciones_agrupadas = transacciones.groupby(interes_group)[valores_medida].sum().reset_index()
transacciones_agrupadas.head(10)

,Codigo_Cliente,DESCRI_PRODUCTO,Categoria_Agrupada,Cantidad,Valor_Total_RD
0,1,VISA EMPRESARIAL,Comida Rapida,1,1179.99
1,1,VISA EMPRESARIAL,Ferreterias y Hogar,2,2114.62
2,1,VISA EMPRESARIAL,Gasolineras y Vehiculos,8,93185.54
3,1,VISA EMPRESARIAL,Hoteles Moteles Resorts,12,1079140.22
4,1,VISA EMPRESARIAL,Licores,8,199882.92
5,1,VISA EMPRESARIAL,Musica,2,19900.00
6,1,VISA EMPRESARIAL,Otros Servicios,64,1563185.57
7,1,VISA EMPRESARIAL,Restaurantes,17,181442.05
8,1,VISA EMPRESARIAL,Ropas,4,57871.96
9,1,VISA EMPRESARIAL,Servicios Agropecuarios,1,5900.00


In [21]:
def get_top_n_comercios_medida(transacciones_agrupadas, medida):
    top_n_comercios_medida = transacciones_agrupadas.sort_values(medida,ascending=False)\
                                                  .groupby(identificador_group)\
                                                  .head(top_n)\
                                                  .sort_values(identificador_group)
    return top_n_comercios_medida
    

In [22]:
top_n_comercios_cantidad = get_top_n_comercios_medida(transacciones_agrupadas,'Cantidad')
top_n_comercios_cantidad.Categoria_Agrupada.value_counts().head(20)

Supermercados y Alimentos                          40865
Gasolineras y Vehiculos                            24508
Servicios de Salud, Belleza y Recreacion           13971
Tienda De Venta Especializada                      12891
Servicios de Comunicacion, Internet y Telecable    11570
Ropas                                               9950
Comida Rapida                                       8584
Servicios Gubernamentales                           6063
Ferreterias y Hogar                                 5772
Restaurantes                                        5363
Servicios de Transporte y Viajes                    2406
Otros Servicios                                     1480
Servicios Educativos                                1379
Licores                                             1262
Musica                                              1241
Instituciones Financieras y Aseguradoras            1040
Ventas y Servicios de Equipos Electricos             836
Vida Nocturna                  

En esta tabla, se muestran la cantidad de clientes que según su total de transacciones, prefieren una categoria de comercio especifica. En esta categoria, se puede visualizar el enfoque en categorias que gastan menos pero que son consumidas más frecuentemente, como *Comida Rapida*, *Musica* y *Vida Nocturna*. 

# Por Monto: Mientras más gasta, mejor

In [23]:
top_n_comercios_valor = get_top_n_comercios_medida(transacciones_agrupadas,'Valor_Total_RD')
top_n_comercios_valor.Categoria_Agrupada.value_counts().head(20)

Supermercados y Alimentos                          38633
Gasolineras y Vehiculos                            21009
Servicios de Comunicacion, Internet y Telecable    13363
Tienda De Venta Especializada                      13359
Ropas                                              12219
Servicios de Salud, Belleza y Recreacion           12044
Ferreterias y Hogar                                 7737
Servicios Gubernamentales                           6740
Servicios de Transporte y Viajes                    4624
Restaurantes                                        4259
Servicios Educativos                                3436
Comida Rapida                                       3117
Otros Servicios                                     2027
Instituciones Financieras y Aseguradoras            1994
Hoteles Moteles Resorts                             1426
Ventas y Servicios de Equipos Electricos            1416
Licores                                             1193
Vida Nocturna                  

A diferencia de las preferencias por Cantidad de transacciones, las categorias en las que se suele gastar más dinero tienen mayor preferencia. Tambien, hay mucha mayor participación de categorias como *Viajes*, *Hogar* y *Servicios Educativos*.

# Por Recencia: Mientras más reciente, mejor

In [24]:
import gc
gc.collect()

70

In [25]:
def crear_recencia(transacciones):
    recencia = pd.qcut(transacciones.Meses_Anteriores,10,labels=list(range(10,0,-1))).astype(int)
    return recencia

Definimos la recencia en 10 valores distintos, según los meses anteriores a la última transacción de la categoria. Los rangos de valores se muestran a continuación.

In [26]:
pd.qcut(transacciones.Meses_Anteriores,10).cat.categories

IntervalIndex([(-0.001, 1.0], (1.0, 2.0], (2.0, 3.0], (3.0, 4.0], (4.0, 5.0], (5.0, 6.0], (6.0, 8.0], (8.0, 9.0], (9.0, 10.0], (10.0, 11.0]],
              closed='right',
              dtype='interval[float64]')

Estos valores fueron reemplazados con los numeros del 10 al 1, siendo 10 el más reciente (menos de un mes) y 1 el más antiguo.

In [27]:
transacciones['Recencia'] = crear_recencia(transacciones)
transacciones.Recencia.describe().astype(int)

count    1981931
mean           5
std            2
min            1
25%            4
50%            6
75%            9
max           10
Name: Recencia, dtype: int32

In [28]:
def get_top_n_comercios_recencia(transacciones):
    
    top_n_comercios_recencia= transacciones.sort_values(by=['Codigo_Cliente','Recencia'],ascending=[True,False]
                                 ).groupby(interes_group + ['Recencia'])\
                                 ['Cantidad'].count()\
                                 .reset_index()\
                                 .sort_values(by=['Recencia','Cantidad'],ascending=[False,False])\
                                 .groupby(identificador_group)\
                                 .head(top_n)\
                                 .sort_values(identificador_group)
                                                
    return top_n_comercios_recencia
    

In [29]:
top_n_comercios_recencia = get_top_n_comercios_recencia(transacciones)
top_n_comercios_recencia.Categoria_Agrupada.value_counts().head(20)

Supermercados y Alimentos                          38126
Gasolineras y Vehiculos                            26663
Servicios de Comunicacion, Internet y Telecable    12338
Servicios de Salud, Belleza y Recreacion           12138
Tienda De Venta Especializada                      11990
Ropas                                              10995
Comida Rapida                                      10149
Ferreterias y Hogar                                 6921
Servicios Gubernamentales                           6712
Restaurantes                                        5705
Servicios de Transporte y Viajes                    2504
Otros Servicios                                     1947
Servicios Educativos                                1683
Musica                                              1630
Instituciones Financieras y Aseguradoras            1570
Licores                                             1560
Hoteles Moteles Resorts                              994
Ventas y Servicios de Equipos E

En este metodo, se ve mayor presencia en categorias recurrentes, como *Musica*, *Gasolineras* y *Cine y Peliculas*. 

# Termometro: Una combinación estadistica de las tres anteriores.

En este metodo, se utilizan tecnicas matematicas y estadisticas para llevar la Cantidad y Monto de transacciones a la misma escala (ya que una usualmente se mide del 1 al 100, y la otra en millones), combinarlas de tal manera que ambas tengan el mismo peso y por último multiplicar el valor resultante por la recencia. De esta manera se combinan los 3 metodos anteriores de manera estadistica.

In [30]:
def crear_termometro(value_matrix):
    value_matrix = MinMaxScaler().fit_transform(value_matrix)
    termometro = PCA(n_components=1).fit_transform(value_matrix)
    termometro_scaled = MinMaxScaler().fit_transform(termometro)
    return termometro_scaled

In [31]:
transacciones_agrupadas['Recencia'] = transacciones.groupby(interes_group)['Recencia'].max().reset_index()['Recencia']

Ejemplo de la combinación creada por el Termometro.

In [32]:
transacciones_agrupadas['Termometro'] = crear_termometro(transacciones_agrupadas[['Cantidad','Valor_Total_RD']].values)
transacciones_agrupadas['Termometro'] = transacciones_agrupadas['Termometro'] * transacciones_agrupadas.Recencia
transacciones_agrupadas.sort_values(by='Termometro',ascending=False).head(10)

,Codigo_Cliente,DESCRI_PRODUCTO,Categoria_Agrupada,Cantidad,Valor_Total_RD,Recencia,Termometro
110153,163427,VISA EMPRESARIAL,Supermercados y Alimentos,6176,7273634.25,10,10.000000
110143,163427,VISA EMPRESARIAL,Restaurantes,3779,2631648.76,10,6.019038
110136,163427,VISA EMPRESARIAL,Comida Rapida,3091,1346483.97,10,4.878975
136062,203354,VISA EMPRESARIAL,Ferreterias y Hogar,40,77518352.00,10,4.288449
110139,163427,VISA EMPRESARIAL,Hoteles Moteles Resorts,1503,2449575.31,10,2.469494
32846,58655,VISA ORO,Tienda De Venta Especializada,1414,2969478.50,10,2.359437
41806,74414,VISA INFINITE,Ropas,98,32134130.00,10,1.903426
52100,87479,VISA INFINITE,Servicios de Transporte y Viajes,848,7916723.44,10,1.749019
100652,151842,VISA EMPRESARIAL,Servicios de Transporte y Viajes,14,28069550.00,10,1.551111
12,1,VISA EMPRESARIAL,"Servicios de Comunicacion, Internet y Telecable",171,22761574.59,10,1.505783


In [33]:
top_n_comercios_termometro = get_top_n_comercios_medida(transacciones_agrupadas,'Termometro')
top_n_comercios_termometro.Categoria_Agrupada.value_counts().head(20)

Supermercados y Alimentos                          40974
Gasolineras y Vehiculos                            23940
Servicios de Salud, Belleza y Recreacion           13821
Tienda De Venta Especializada                      13241
Servicios de Comunicacion, Internet y Telecable    12100
Ropas                                              10217
Comida Rapida                                       7381
Servicios Gubernamentales                           6177
Ferreterias y Hogar                                 5817
Restaurantes                                        5031
Servicios de Transporte y Viajes                    2751
Servicios Educativos                                1645
Otros Servicios                                     1457
Licores                                             1225
Musica                                              1206
Instituciones Financieras y Aseguradoras            1178
Ventas y Servicios de Equipos Electricos             962
Vida Nocturna                  

Al combinar las metricas, se nota más la presencia de categorias en la que se gasta mucho, se compra mucho, y se va frecuente, como *Supermercados*, *Salud y Belleza*, *Tiendas de Venta Especializadas (aquí caen muchos negocios en Linea como Amazon)*. 

# RFM: Combinación más heuristica de la Recencia, Cantidad y Valor Monetario

Otro metodo parecido al del termometro es el del RFM (Recency, Frequency, Monetary Value). Igual, consideramos las 3 metricas con el mismo peso, pero estos pesos seran llevados a numeros del 1 al 10, como se hizo con la recencia. Las divisiones se encontraran segun los deciles o aproximaciones de los deciles de la distribucion de cada metrica, igual que como se hizo con la recencia.

División de las cantidades

In [34]:
 pd.cut(transacciones_agrupadas.Cantidad,[0,1,3,5,7,10,20,30,50,100,100000]).cat.categories

IntervalIndex([(0, 1], (1, 3], (3, 5], (5, 7], (7, 10], (10, 20], (20, 30], (30, 50], (50, 100], (100, 100000]],
              closed='right',
              dtype='interval[int64]')

División del Valor Monetario

In [35]:
pd.qcut(transacciones_agrupadas.Valor_Total_RD,10,).cat.categories

IntervalIndex([(0.499, 600.0], (600.0, 1150.0], (1150.0, 1896.01], (1896.01, 2950.0], (2950.0, 4487.0], (4487.0, 6870.0], (6870.0, 10909.088], (10909.088, 18799.206], (18799.206, 39246.004], (39246.004, 77518352.0]],
              closed='right',
              dtype='interval[float64]')

Por último, luego de tener el score RFM de las 3 metricas, se calcula el promedio de las 3 para llegar al score RFM final.

In [36]:
def crear_score_RFM(transacciones_agrupadas):
    transacciones_agrupadas['Cantidad_RFM'] = pd.cut(transacciones_agrupadas.Cantidad,[0,1,3,5,7,10,20,30,50,100,100000]
                                                 ,labels=list(range(1,11))).astype(int)
    transacciones_agrupadas['Valor_Monetario_RFM'] = pd.qcut(transacciones_agrupadas.Valor_Total_RD
                                                         ,10,labels=list(range(1,11))).astype(int)
    transacciones_agrupadas['Score_RFM'] = transacciones_agrupadas[['Recencia','Cantidad_RFM','Valor_Monetario_RFM']
                                                              ].mean(axis=1)
    return transacciones_agrupadas['Score_RFM']

Ejemplo de algunos valores del score RFM en los datos.

In [37]:
transacciones_agrupadas['Score_RFM'] = crear_score_RFM(transacciones_agrupadas)
transacciones_agrupadas[['Codigo_Cliente','Categoria_Agrupada',
                         'Cantidad_RFM','Recencia','Valor_Monetario_RFM','Score_RFM']].sample(10)

,Codigo_Cliente,Categoria_Agrupada,Cantidad_RFM,Recencia,Valor_Monetario_RFM,Score_RFM
286862,365702,Tienda De Venta Especializada,3,8,5,5.333333
359830,423193,Supermercados y Alimentos,7,10,8,8.333333
252214,341654,Gasolineras y Vehiculos,6,10,4,6.666667
147155,218520,Hoteles Moteles Resorts,2,6,8,5.333333
221832,315573,Articulos de Oficina,5,10,10,8.333333
60455,98043,Otros Servicios,1,4,3,2.666667
160608,236974,Licores,2,4,5,3.666667
135271,202304,Supermercados y Alimentos,6,10,9,8.333333
146515,217606,Supermercados y Alimentos,4,6,8,6.000000
293180,370202,Servicios Educativos,1,7,4,4.000000


In [38]:
top_n_comercios_rfm = get_top_n_comercios_medida(transacciones_agrupadas,'Score_RFM')
top_n_comercios_rfm.Categoria_Agrupada.value_counts().head(20)

Supermercados y Alimentos                          40349
Gasolineras y Vehiculos                            23143
Tienda De Venta Especializada                      13557
Servicios de Salud, Belleza y Recreacion           13378
Servicios de Comunicacion, Internet y Telecable    12805
Ropas                                              11104
Servicios Gubernamentales                           6505
Ferreterias y Hogar                                 6408
Comida Rapida                                       5278
Restaurantes                                        4984
Servicios de Transporte y Viajes                    3102
Servicios Educativos                                2096
Otros Servicios                                     1609
Instituciones Financieras y Aseguradoras            1337
Licores                                             1263
Ventas y Servicios de Equipos Electricos            1086
Musica                                               965
Hoteles Moteles Resorts        

Con esta metrica, se obtienen valores similares al del termometro, con un poco de variación en algunas categorias. Esta metrica es la que será utilizada como la metrica final para determinar las preferencias de comercios.

# Exploración

## Variación entre Metricas

Según vimos, entre cada metrica existe una cantidad diferente de clientes en cada categoria. Analizamos esta variación para determinar la causa de estas diferencias.

In [39]:
datasets_top_n = {'Cantidad':top_n_comercios_cantidad,
                  'Recencia': top_n_comercios_recencia,
                  'Termometro':top_n_comercios_termometro,
                  'Valor':top_n_comercios_valor,
                  'RFM': top_n_comercios_rfm}
len(datasets_top_n)

5

In [40]:
def create_value_counts_dataframe(datasets):
    if len(categoria_comercio_interes) < 2:
        value_counts = list(datasets.values())[0][categoria_comercio_interes[0]].value_counts()
        cats = value_counts.index
        value_counts_df = pd.DataFrame({
            **{f"Preferencias_{k}":top_n[categoria_comercio_interes[0]].value_counts()[cats] for k,top_n in datasets.items()}
        })
        return value_counts_df
    else:
        value_counts = list(datasets.values())[0].groupby(categoria_comercio_interes[:2]
                                                         )['Cantidad'].count()
        cats = value_counts.index
        value_counts_df = pd.DataFrame({
            **{f"Preferencias_{k}":top_n.groupby(categoria_comercio_interes[:2]
                                                         )['Cantidad'].count()[cats] 
               for k,top_n in datasets.items()}
        })
        return value_counts_df

Agrupamos la cantidad de clientes por producto por categoria por metrica, para luego crear estadisticas con esta, como el promedio de clientes, la desviación estandar y  el porcentaje de variación por categoria.

In [41]:
value_counts_df = create_value_counts_dataframe(datasets_top_n)
value_counts_df.head()

Preferencias_Cantidad  \
DESCRI_PRODUCTO Categoria_Agrupada                               
CREDITO JOVEN   Articulos de Oficina                         6   
                Cine y Peliculas                            47   
                Comida Rapida                              560   
                Ferreterias y Hogar                         94   
                Gasolineras y Vehiculos                    702   

                                         Preferencias_Recencia  \
DESCRI_PRODUCTO Categoria_Agrupada                               
CREDITO JOVEN   Articulos de Oficina                        12   
                Cine y Peliculas                            55   
                Comida Rapida                              555   
                Ferreterias y Hogar                        116   
                Gasolineras y Vehiculos                    846   

                                         Preferencias_Termometro  \
DESCRI_PRODUCTO Categoria_Agrupada                                 
CREDITO JOVEN   Articulos de Oficina                           3   
                Cine y Peliculas                              34   
                Comida Rapida                                510   
                Ferreterias y Hogar                          103   
                Gasolineras y Vehiculos                      694   

                                         Preferencias_Valor  Preferencias_RFM  
DESCRI_PRODUCTO Categoria_Agrupada                                             
CREDITO JOVEN   Articulos de Oficina                    7.0                 5  
                Cine y Peliculas                       24.0                29  
                Comida Rapida                         279.0               412  
                Ferreterias y Hogar                   140.0               109  
                Gasolineras y Vehiculos               620.0               674

In [42]:
preferencias_counts_cols = [col for col in value_counts_df if 'Preferencias' in col]
value_counts_df['Promedio'] = value_counts_df[preferencias_counts_cols].mean(axis=1).astype(int)
value_counts_df['Desviacion'] = value_counts_df[preferencias_counts_cols].std(axis=1)
value_counts_df["Porcentaje_Variacion (%)"] = ((value_counts_df["Desviacion"] 
                                            / value_counts_df['Promedio'])* 100).round(2)
value_counts_df = value_counts_df.reset_index()

In [43]:
value_counts_df.head()

,DESCRI_PRODUCTO,Categoria_Agrupada,Preferencias_Cantidad,Preferencias_Recencia,Preferencias_Termometro,Preferencias_Valor,Preferencias_RFM,Promedio,Desviacion,Porcentaje_Variacion (%)
0,CREDITO JOVEN,Articulos de Oficina,6,12,3,7.0,5,6,3.361547,56.03
1,CREDITO JOVEN,Cine y Peliculas,47,55,34,24.0,29,37,12.872451,34.79
2,CREDITO JOVEN,Comida Rapida,560,555,510,279.0,412,463,118.889444,25.68
3,CREDITO JOVEN,Ferreterias y Hogar,94,116,103,140.0,109,112,17.415510,15.55
4,CREDITO JOVEN,Gasolineras y Vehiculos,702,846,694,620.0,674,707,83.923775,11.87


In [44]:
def bar_plotly(dataframe,dim,value,title,show_text=False):   
    sorted_dataframe = dataframe.sort_values(by=value,ascending=False)
    data = [go.Bar(
                y=sorted_dataframe[value],
                x=sorted_dataframe[dim],
                marker=dict(
                color='#1D62AE',
                line=dict(
                    color='#14A751',
                    width=1,
                )
            )
        )]
    
    if show_text:
        data = [go.Bar(
                y=sorted_dataframe[value],
                x=sorted_dataframe[dim],
                marker=dict(
                color='#1D62AE',
                line=dict(
                    color='#14A751',
                    width=1,
                )
            )
        )]

    layout = go.Layout(
        title=title,
        width=800,
        height=400,
         xaxis=dict(
            title=dim,
            titlefont=dict(
                family='Myriad',
                size=20,
                color='#14A751'
            ),
             showticklabels=show_text
        ),
         yaxis=dict(
            title=value,
            titlefont=dict(
                family='Myriad',
                size=20,
                color='#14A751'
            )
        )
    )
    fig = go.Figure(data=data, layout=layout)
    ipy(fig, config=plotly_config)

In [45]:
def create_grouping(data,groupby_cols,value_cols,new_cols,aggregation="sum"):
    grouped_data = data.groupby(groupby_cols)[value_cols].agg(aggregation).reset_index()
    grouped_data.columns = new_cols
    return grouped_data

def crear_estadisticas_value_counts(value_count_df,count_cols,grouping_col = "Categoria_Agrupada"):
    preferencias_total = value_count_df.groupby(grouping_col)[count_cols].sum()
    preferencias_total["Promedio"] = preferencias_total[count_cols].mean(axis=1)
    preferencias_total["Desviacion"] = preferencias_total[count_cols].std(axis=1)
    preferencias_total["Porcentaje_Variacion (%)"] = ((preferencias_total["Desviacion"] / preferencias_total["Promedio"]
                                                      ) * 100).round(2)
    return preferencias_total.reset_index()

In [46]:
producto_desviacion_promedio = create_grouping(value_counts_df,["DESCRI_PRODUCTO"]
                                               ,"Porcentaje_Variacion (%)"
                                               , ["DESCRI_PRODUCTO","mean_Porcentaje_Variacion (%)"] 
                                               ,["mean"])
producto_desviacion_promedio

,DESCRI_PRODUCTO,mean_Porcentaje_Variacion (%)
0,CREDITO JOVEN,22.257500
1,MULTICREDITO,12.588077
2,VISA CLASICA,17.927308
3,VISA EMPRESARIAL,28.032800
4,VISA INFINITE,28.719231
5,VISA ORO,20.717308
6,VISA PLATINO,25.761154


A continuación, algunas graficas para ilustrar esta variación. Variación alta significa que para una metrica, una categoria tiene una cantidad de clientes mucho mayor/ mucho menor, usualmente por una diferencia alta en Cantidad o Monto.

In [47]:
bar_plotly(dataframe=producto_desviacion_promedio,dim="DESCRI_PRODUCTO",value="mean_Porcentaje_Variacion (%)"
           ,title="Variación de Preferencia Promedio por Producto",show_text=True)

In [48]:
categoria_value_counts = crear_estadisticas_value_counts(value_counts_df, preferencias_counts_cols
                                                         ,grouping_col = "Categoria_Agrupada")
categoria_value_counts.head()

,Categoria_Agrupada,Preferencias_Cantidad,Preferencias_Recencia,Preferencias_Termometro,Preferencias_Valor,Preferencias_RFM,Promedio,Desviacion,Porcentaje_Variacion (%)
0,Articulos de Oficina,274,443,282,379.0,275,330.6,76.865467,23.25
1,Cine y Peliculas,472,654,350,214.0,258,389.6,177.777389,45.63
2,Comida Rapida,8584,10149,7381,3117.0,5278,6901.8,2764.289004,40.05
3,Ferreterias y Hogar,5772,6921,5817,7737.0,6408,6531.0,822.730515,12.60
4,Gasolineras y Vehiculos,24508,26663,23940,21009.0,23143,23852.6,2057.008094,8.62


In [49]:
bar_plotly(dataframe=categoria_value_counts,dim="Categoria_Agrupada",value="Porcentaje_Variacion (%)"
           ,title="Variación de Preferencia por Categoria",show_text=True)

Las categorias de mayor variación son por altas cantidades o recencia y montos bajos, como *Cine y Peliculas*, *Comida Rapida*, o viceversa como *Servicios Educativos* y *Servicios de Construcción*.  Son más volatiles a la hora de determinar la preferencia, por esto se usa una combinación de las metricas.

## Promedio Preferencias por Categoria

A continuación, fueron creados algunos graficos para visualizar las preferencias por categoria y tipo de tarjeta. Las cantidades fueron determinadas según el promedio de cantidad de clientes entre metricas. Aunque no puede ser usado para preferencias puntuales a clientes, si pueden ser utilizados con fines de análisis.

In [50]:
bar_plotly(dataframe=categoria_value_counts,dim="Categoria_Agrupada",value="Promedio"
           ,title="Promedio de Preferencias de Clientes por Categoria",show_text=True)

Las categorias más populares son **Supermercados y Alimentos** y **Gasolineras y Vehiculos**. Estas categorias engloban necesidades del día a día de casi todo el mundo. Otra categorias populares son **Tienda De Venta Especializada** que captura las compras en línea como en Amazon y **Servicios de Salud, Belleza y Recreación** que incluye servicios de salud como clinicas, gimnasios, salones de belleza y más. Veamos estos promedios por tipo de tarjeta:

In [51]:
tipos_tarjetas = ["Visa Clasica","Visa Oro","Visa Platino","Visa Infinite","Visa Empresarial", "Credito Joven",
                 "Multicredito"]
tipos_tarjetas_upper = [t.upper() for t in tipos_tarjetas]
plot_titles = ["Porcentaje de Pertenencia por Categoria," + t for t in tipos_tarjetas]

In [52]:
def bar_plotly_categoria_tipo_tarjeta(value_counts_df,tipo_tarjeta,plot_col,plot_title):
    count_tarjeta = value_counts_df.loc[value_counts_df.DESCRI_PRODUCTO == tipo_tarjeta]
    bar_plotly(dataframe=count_tarjeta,dim="Categoria_Agrupada",value=plot_col
           ,title=plot_title,show_text=True)

In [53]:
@interact 
def bar_promedio_preferencia_tarjeta(tipo_tarjeta=tipos_tarjetas_upper):
    bar_plotly_categoria_tipo_tarjeta(value_counts_df,tipo_tarjeta,"Promedio","Promedio de Preferencia por Categoria")

interactive(children=(Dropdown(description='tipo_tarjeta', options=('VISA CLASICA', 'VISA ORO', 'VISA PLATINO'…

## Porcentaje de Pertenencia de Preferencias Por Categoria Por Tarjeta

Para distinguir mejor las diferencias entre tipos de tarjeta, tambien fue calculado el porcentaje de las preferencias de cada categoria por tipo de tarjeta. Esto ayuda a determinar categorias sobresalientes en cada tipo de tarjeta.

In [54]:
value_counts_df = value_counts_df.sort_values(by="Categoria_Agrupada").set_index("Categoria_Agrupada")
value_counts_categoria_total_promedio = value_counts_df.groupby("Categoria_Agrupada")["Promedio"].sum()
value_counts_categoria_total_promedio
value_counts_df["Porcentaje_Total_Categoria (%)"] = (value_counts_df["Promedio"] 
                                                              / value_counts_categoria_total_promedio * 100
                                                    ).round(2)
value_counts_df = value_counts_df.reset_index()
value_counts_df.head()

,Categoria_Agrupada,DESCRI_PRODUCTO,Preferencias_Cantidad,Preferencias_Recencia,Preferencias_Termometro,Preferencias_Valor,Preferencias_RFM,Promedio,Desviacion,Porcentaje_Variacion (%),Porcentaje_Total_Categoria (%)
0,Articulos de Oficina,CREDITO JOVEN,6,12,3,7.0,5,6,3.361547,56.03,1.83
1,Articulos de Oficina,VISA EMPRESARIAL,14,11,12,13.0,8,11,2.302173,20.93,3.36
2,Articulos de Oficina,VISA ORO,42,75,41,64.0,40,52,16.102795,30.97,15.90
3,Articulos de Oficina,VISA CLASICA,131,204,123,160.0,127,149,34.022052,22.83,45.57
4,Articulos de Oficina,VISA PLATINO,42,84,57,87.0,58,65,19.269146,29.64,19.88


In [55]:
@interact
def bar_porcentaje_categoria_tarjeta(tipos_tarjetas_upper=tipos_tarjetas_upper):
    bar_plotly_categoria_tipo_tarjeta(value_counts_df,tipos_tarjetas_upper,"Porcentaje_Total_Categoria (%)"
                                  ,"Porcentaje de Pertenecia por Categoria")

interactive(children=(Dropdown(description='tipos_tarjetas_upper', options=('VISA CLASICA', 'VISA ORO', 'VISA …

# Asignación de Preferencias por Cliente

Como fue mencionado anteriormente, se utilizó la metrica del RFM para determinar las categorias de comercio finales preferentes de un cliente. Esta vez, se ignora totalmente el tipo de producto, y se concentra totalmente en el cliente.

In [56]:
def conseguir_top_n_comercios_por_cliente_rfm(transacciones,
                                                   interes_group=["Codigo_Cliente","Categoria_Agrupada"]):
    transacciones_agrupadas = transacciones.groupby(interes_group)[valores_medida].sum().reset_index()
    transacciones_agrupadas['Recencia'] = transacciones.groupby(interes_group)['Recencia'].max().reset_index()['Recencia']
    transacciones_agrupadas['Score_RFM'] = crear_score_RFM(transacciones_agrupadas) 
    top_n_comercios_rfm = get_top_n_comercios_medida(transacciones_agrupadas,'Score_RFM')
    return top_n_comercios_rfm



In [57]:
identificador_group = ['Codigo_Cliente']
top_n_comercios_rfm_cliente = conseguir_top_n_comercios_por_cliente_rfm(transacciones)
qgrid_preferencias_cliente = qgrid.show_grid(top_n_comercios_rfm_cliente[['Codigo_Cliente','Categoria_Agrupada',
                                                                         'Score_RFM']])

In [58]:
qgrid_preferencias_cliente

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Visualización RFM

Con los pesos de Recencia, Frecuencia y Valor Monetario de las categorias de comercio, procedimos a crear visualizaciones que mostraran, tanto para los comercios como para los clientes, su valor completo para el negocio según su transaccionalidad por tarjeta. Primero, RFM por categoria de Comercio.

In [59]:
rfm_stats = transacciones_agrupadas.groupby('Categoria_Agrupada'
                               )[['Cantidad_RFM','Valor_Monetario_RFM','Recencia']].mean()
rfm_stats['Cantidad_Clientes'] = top_n_comercios_rfm_cliente.groupby('Categoria_Agrupada'
                                                                    )['Codigo_Cliente'
                                                                     ].count()
rfm_stats['Cantidad_Clientes'] = rfm_stats['Cantidad_Clientes'].fillna(0)
rfm_stats = rfm_stats.reset_index()
rfm_stats.head()

,Categoria_Agrupada,Cantidad_RFM,Valor_Monetario_RFM,Recencia,Cantidad_Clientes
0,Alquiler de Automoviles,1.133333,7.133333,6.133333,0.0
1,Articulos de Oficina,2.252874,4.905492,6.708812,261.0
2,Cine y Peliculas,1.889518,2.436084,6.291254,247.0
3,Comida Rapida,2.913375,3.694305,7.391586,4971.0
4,Ferreterias y Hogar,2.304384,5.746752,7.114073,5904.0


In [60]:
rfm_stats['Texto_Plot'] = ("Categoria:" + rfm_stats['Categoria_Agrupada'].astype(str) + "<br>" + 
                           "RFM Cantidad:" + rfm_stats['Cantidad_RFM'].round(2).astype(str)  + "<br>" + 
                           "RFM Recencia:" + rfm_stats['Recencia'].round(2).astype(str)  + "<br>" + 
                           "RFM Valor Monetario:" + rfm_stats['Valor_Monetario_RFM'].round(2).astype(str)  + "<br>" + 
                           "Clientes:" + rfm_stats['Cantidad_Clientes'].astype(int).astype(str))
rfm_stats.head()

,Categoria_Agrupada,Cantidad_RFM,Valor_Monetario_RFM,Recencia,Cantidad_Clientes,Texto_Plot
0,Alquiler de Automoviles,1.133333,7.133333,6.133333,0.0,Categoria:Alquiler de Automoviles<br>RFM Canti...
1,Articulos de Oficina,2.252874,4.905492,6.708812,261.0,Categoria:Articulos de Oficina<br>RFM Cantidad...
2,Cine y Peliculas,1.889518,2.436084,6.291254,247.0,Categoria:Cine y Peliculas<br>RFM Cantidad:1.8...
3,Comida Rapida,2.913375,3.694305,7.391586,4971.0,Categoria:Comida Rapida<br>RFM Cantidad:2.91<b...
4,Ferreterias y Hogar,2.304384,5.746752,7.114073,5904.0,Categoria:Ferreterias y Hogar<br>RFM Cantidad:...


In [61]:
def crear_scatter_RFM(data,title,size=None):
    trace1 = go.Scatter(
    x= data.Cantidad_RFM,
    y = data.Recencia,
    mode='markers',
    text=data.Texto_Plot,
    marker=dict(
        size=  np.log1p(data[size]) * 4 if size else 16,
        color = data.Valor_Monetario_RFM, 
        colorbar=dict(
                title='RFM Valor Monetario'
            ),
        colorscale='Viridis',
        showscale=True
        )
    )

    layout = go.Layout(
            title=title,
        hovermode = 'closest',
            autosize=True,
            
             xaxis=dict(
                title="RFM Cantidad",
                titlefont=dict(
                    family='Myriad',
                    size=20,
                    color='#14A751'
                ),
            ),
             yaxis=dict(
                 title="RFM Recencia",
                titlefont=dict(
                    family='Myriad',
                    size=20,
                    color='#14A751'
                )
            )
        )

    data = [trace1]
    fig = go.Figure(data=data, layout=layout)
    ipy(fig)

**Nota**: En estos graficos, el eje horizontal representa la cantidad de transacciones pero reducida a su valor RFM, y el eje vertical representa la recencia, mientras más alto más reciente fue la última transacción. Los colores representan el valor monetario, mientras más intenso, mayor monto promedio fue transaccionado. Por último, el tamaño de las burbujas representa la cantidad de clientes que caen en esta categoria.

In [62]:
crear_scatter_RFM(rfm_stats,"RFM por Categoria de Comercio",size='Cantidad_Clientes')

Como podemos observar, existe una relación entre las 3 metricas, en el que la mayoria de las veces, cuando una aumenta las otras tambien. Por ejemplo, la categoria de **Supermercados y Alimentos** tiene un valor monetario, recencia y frecuencia altos. Pero, esta relación no es totalmente lineal, como en la categoria de **Instituciones Financieras** con alto valor monetario y Recencia, pero cantidad baja. **En sí, se puede visualizar una relación lineal entre las metricas, siendo las categorias de comercio más comunes las de más alto valor RFM**.

Ahora visualicemos lo mismo, pero ahora según la cantidad de clientes que caen en una combinación de Recencia y Frecuencia de RFM.

In [63]:
transacciones_agrupadas_cliente = transacciones.groupby('Codigo_Cliente')[['Cantidad','Valor_Total_RD']].sum()
transacciones_agrupadas_cliente['Recencia'] = transacciones.groupby('Codigo_Cliente')['Meses_Anteriores'].min()
transacciones_agrupadas_cliente.head()

,Cantidad,Valor_Total_RD,Recencia
Codigo_Cliente,,,
1,716,51700362.12,0
10,63,91663.73,0
22,62,137621.64,0
51,154,259025.72,0
59,360,877717.21,0


In [64]:
transacciones_agrupadas_cliente['Cantidad_RFM'] = pd.qcut(transacciones_agrupadas_cliente.Cantidad,5,
                                                         labels=list(range(1,6))).astype(int)
transacciones_agrupadas_cliente['Valor_Monetario_RFM'] = pd.qcut(transacciones_agrupadas_cliente.Valor_Total_RD,5,
                                                         labels=list(range(1,6))).astype(int)

transacciones_agrupadas_cliente['Recencia'] = pd.cut(transacciones_agrupadas_cliente.Recencia
                                                     ,[-0.01,1,3,6,9,12],labels=list(range(5,0,-1))).astype(int)

transacciones_agrupadas_cliente['Score_RFM'] = transacciones_agrupadas_cliente[['Recencia'
                                                                                ,'Cantidad_RFM'
                                                                                ,'Valor_Monetario_RFM']].mean(axis=1)
transacciones_agrupadas_cliente=transacciones_agrupadas_cliente.reset_index()
transacciones_agrupadas_cliente.head()

,Codigo_Cliente,Cantidad,Valor_Total_RD,Recencia,Cantidad_RFM,Valor_Monetario_RFM,Score_RFM
0,1,716,51700362.12,5,5,5,5.000000
1,10,63,91663.73,5,4,4,4.333333
2,22,62,137621.64,5,4,4,4.333333
3,51,154,259025.72,5,5,5,5.000000
4,59,360,877717.21,5,5,5,5.000000


In [65]:
transacciones_agrupadas_RFM = transacciones_agrupadas_cliente.groupby(
        ['Recencia','Cantidad_RFM'])['Valor_Monetario_RFM'].mean().reset_index()
transacciones_agrupadas_RFM['Cantidad_Clientes'] = transacciones_agrupadas_cliente.groupby(
        ['Recencia','Cantidad_RFM'])['Codigo_Cliente'].count().reset_index()['Codigo_Cliente']
transacciones_agrupadas_RFM = transacciones_agrupadas_RFM.rename(columns={'Codigo_Cliente':'Cantidad_Clientes'})
transacciones_agrupadas_RFM.head()

,Recencia,Cantidad_RFM,Valor_Monetario_RFM,Cantidad_Clientes
0,1,1,1.190237,1987
1,1,2,1.911932,352
2,1,3,2.652542,118
3,1,4,3.772727,44
4,1,5,4.600000,10


In [66]:
transacciones_agrupadas_RFM['Texto_Plot'] = (
                           "RFM Cantidad:" + transacciones_agrupadas_RFM['Cantidad_RFM'].round(2).astype(str)  + "<br>" + 
                           "RFM Recencia:" + transacciones_agrupadas_RFM['Recencia'].round(2).astype(str)  + "<br>" + 
                           "RFM Valor Monetario Promedio:" + transacciones_agrupadas_RFM['Valor_Monetario_RFM'
                                                                                   ].round(2).astype(str) + "<br>" + 
                           "Cantidad Clientes:" + transacciones_agrupadas_RFM['Cantidad_Clientes'].astype(str) 
)
transacciones_agrupadas_RFM.head()

,Recencia,Cantidad_RFM,Valor_Monetario_RFM,Cantidad_Clientes,Texto_Plot
0,1,1,1.190237,1987,RFM Cantidad:1<br>RFM Recencia:1<br>RFM Valor ...
1,1,2,1.911932,352,RFM Cantidad:2<br>RFM Recencia:1<br>RFM Valor ...
2,1,3,2.652542,118,RFM Cantidad:3<br>RFM Recencia:1<br>RFM Valor ...
3,1,4,3.772727,44,RFM Cantidad:4<br>RFM Recencia:1<br>RFM Valor ...
4,1,5,4.600000,10,RFM Cantidad:5<br>RFM Recencia:1<br>RFM Valor ...


In [67]:
crear_scatter_RFM(transacciones_agrupadas_RFM,"RFM por Cliente", size='Cantidad_Clientes')

Para los clientes, se puede apreciar que la mayoría caen en segmentos donde su transacción más reciente fue en el ultimo mes, por lo tanto son usuarios activos de la tarjeta. Sin embargo, una gran parte de los clientes (aproximadamente 10,000), tienen una cantidad y valor monetario bajos en su tarjeta. Por otro lado, hay un número de clientes (aproximadamente 1,500) con un nivel bueno de transaccionalidad pero baja recencia, que pueden ser "rescatados". **Esta gráfica nos ayuda a visualizar a los clientes de TC según segmentos de uso de la misma!**

## Exploración de Clientes

A continuación, se calcularon las top 3 categorias de comercio preferentes por cliente, utilizando la metrica del RFM, y se realizó un análisis según caracteristicas de los clientes. Primero, el listado de preferencias por cliente.

In [68]:
data_clientes_bsc = pd.read_csv("data/data_clientes_bsc.csv")
data_clientes_bsc.head()

,Codigo_Cliente,Tipo_De_Persona,Segmento,Edad,Zona,Sexo,Limite_TC
0,10.0,N,Clasico,36.0,Norte,F,50000.0
1,22.0,N,Emergente,54.0,Metropolitana,F,84000.0
2,51.0,N,Clasico,56.0,Norte,F,72000.0
3,59.0,N,Preferencial,49.0,Norte,M,100000.0
4,60.0,N,Emergente,57.0,Norte,M,248000.0


In [69]:
preferencias_clientes_full = top_n_comercios_rfm_cliente.merge(data_clientes_bsc,how='left')
preferencias_clientes_qgrid = qgrid.show_grid(preferencias_clientes_full[['Codigo_Cliente','Categoria_Agrupada',
                                                          'Score_RFM','Cantidad','Valor_Total_RD',
                                                          'Segmento','Sexo','Edad']])
preferencias_clientes_full.head()

,Codigo_Cliente,Categoria_Agrupada,Cantidad,Valor_Total_RD,Recencia,Cantidad_RFM,Valor_Monetario_RFM,Score_RFM,Tipo_De_Persona,Segmento,Edad,Zona,Sexo,Limite_TC
0,1,Servicios de Transporte y Viajes,73,3025450.38,10,9,10,9.666667,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Otros Servicios,64,1563185.57,10,9,10,9.666667,NaN,NaN,NaN,NaN,NaN,NaN
2,1,"Servicios de Comunicacion, Internet y Telecable",171,22761574.59,10,10,10,10.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,10,"Servicios de Salud, Belleza y Recreacion",6,6805.00,10,4,6,6.666667,N,Clasico,36.0,Norte,F,50000.0
4,10,Ropas,6,5770.00,10,4,6,6.666667,N,Clasico,36.0,Norte,F,50000.0


In [70]:
preferencias_clientes_qgrid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Luego, procedimos a crear visualizaciones que ilustraran las diferencias de preferencias entre distintos tipos de clientes.

#### Edad Mediana por Categoria de Comercio

In [71]:
edad_mediana_por_categoria = create_grouping(preferencias_clientes_full,["Categoria_Agrupada"],["Edad"]
                ,["Categoria_Agrupada","Edad_Promedio"],aggregation="median")

In [72]:
bar_plotly(dataframe=edad_mediana_por_categoria,dim="Categoria_Agrupada",value="Edad_Promedio"
           ,title="Edad Mediana Por Categoria",show_text=True)

Concerniente a la edad, en los jovenes se nota una enorme preferencia en **Entretenimiento**, **Tecnología**, **Vida Nocturna** y **Compras en Línea**; los clientes más mayores, prefieren servicios para mejorar su futuro, tales como **Inversiones en Tierra y Ganado**, **Servicios del Gobierno** y de **Instituciones Financieras** y tambien **Construcción** y el **Hogar**.

Ahora veamos una distribución según subsegmento de edad (18 a 27: Joven, 28 a 59 Adulto, 60 en Adelante Senior)

In [81]:
def stacked_bar_plotly(dataframe,dim,group,value,title):   
    sorted_dataframe = dataframe.sort_values(by=value,ascending=False)
    data = list()
    group_values = dataframe[group].unique()
    for group_name in group_values:
        group_data = sorted_dataframe.loc[sorted_dataframe[group] == group_name]
        data.append(go.Bar(
                    y=group_data[value],
                    x=group_data[dim],
                    name=group_name
                )
            )
    

    layout = go.Layout(
        title=title,
        height=400,
        width=800,
        barmode='stack',
         xaxis=dict(
            title=dim,
            titlefont=dict(
                family='Myriad',
                size=20,
                color='#14A751'
            ),
        ),
         yaxis=dict(
            title=value,
            titlefont=dict(
                family='Myriad',
                size=20,
                color='#14A751'
            )
        )
    )
    fig = go.Figure(data=data, layout=layout)
    ipy(fig, config=plotly_config)

In [82]:
def create_count_percentage_by_group(dataframe, groupby_cols,count_col
                                     ,new_cols):
    count_data = dataframe.groupby(groupby_cols)[count_col].count()
    percentage_data = count_data.groupby(level=0).apply(lambda x: 100 * x / x.sum()).reset_index()
    percentage_data.columns = new_cols
    return percentage_data

In [75]:
preferencias_clientes_full["Microsegmento_Edad"] = pd.cut(preferencias_clientes_full.Edad
                                                          ,[18,28,59,999],labels=["Joven","Adulto","Senior"])
preferencias_clientes_full["Rango_Limite_TC"] = pd.cut(preferencias_clientes_full.Limite_TC
                                                          ,[0,25000,50000,100000,200000,500000,10000000]
                                                      ,labels=["0 a 25k","25k a 50k","50k a 100k","100k a 200k",
                                                              "200k a 500k","500k en adelante"])

**Nota:** en los siguientes graficos de barra, el eje vertical representa el porcentaje de clientes, sumando a un 100% siempre. Los colores representan las categorias de interes. Se puede hacer doble clic en una categoriar para filtrar el gráfico por la misma. Esto representaría el porcentaje de una categoria según el filtro, no necesariamente que esta categoria domina al filtro.

In [76]:
columnas_preferencias = ["Microsegmento_Edad","Segmento","Rango_Limite_TC","Sexo","Zona"]

In [77]:
def create_percentage_stacked_chart(column):
    pct_preferencias_cat = create_count_percentage_by_group(preferencias_clientes_full
                                              ,["Categoria_Agrupada",column],["Codigo_Cliente"]
                ,["Categoria_Agrupada",column,"Porcentaje_Clientes (%)"])
    stacked_bar_plotly(pct_preferencias_cat,"Categoria_Agrupada",column
                   ,"Porcentaje_Clientes (%)","Preferencias por Categoria por " + column)

In [83]:
@interact
def porcentaje_categoria_columna(columna=columnas_preferencias):
    create_percentage_stacked_chart(columna)

interactive(children=(Dropdown(description='columna', options=('Microsegmento_Edad', 'Segmento', 'Rango_Limite…

Se pueden apreciar las diferencias entre las categorias por los segmentos de edad donde: 

- Las categorias de Entretenimiento, Comida Rapida, Educación y Tecnología tienen mayor captación entre los jovenes.

- El Cuidado del Hogar y Supermercados, los Viajes y Restaurantes tienen mayor captación entre los adultos.

- Las Instituciones Financieras, y Servicios para Empresas (Gubernamentales, Articulos de Oficina) tienen mayor captación entre los senior.

Se pueden apreciar las diferencias entre los segmentos donde: 

- Las categorias de Entretenimiento, y Compras de Bienes tienen auge en los segmentos más bajos.

- Los categorias de Servicios, Restaurantes y Vacaciones son más populares en los segmentos de renta altos.


Con el limite de TC, se ven preferencias similares a las encontradas en los segmentos. Esto tiene sentido, ya que el poder adquisitivo economico está relacionado al limite otorgado en las tarjetas.

Al analizar por Sexo, se notan las siguientes observaciones: 

- Las categorias de Ropas y Salud,Belleza y Recreación son preferidas por las mujeres.
- La Vida Nocturna, Licores y Vehiculos son preferidas por los hombres.
- Existe parecido en ambos sexos para las categorias de Educación, Supermercados, Comida Rapida, Finanzas entre otros.

Cabe destacar que algunas categorias como Restaurantes, Hoteles, y Viajes, aunque sean dominadas por hombres, la naturaleza del hombre invitar a las mujeres en actividades como esa, sesga los resultados hacia el sexo masculino.

Por Zona, se puede apreciar lo siguiente: 

- Mayor presencia de las categorias de Construcción, Cine, Delivery y Comida Rapida en la zona Metro.
- Mayor presencia de las categorias de Equipos Electricos y Agropecuarios en la zona Norte.
- Igual presencia en las demás, con un poco más de peso en la zona Metro.

In [79]:
def create_dim_percentage_stacked_chart(column):
    pct_preferencias_cat = create_count_percentage_by_group(preferencias_clientes_full
                                              ,[column,"Categoria_Agrupada"],["Codigo_Cliente"]
                ,[column,"Categoria_Agrupada","Porcentaje_Clientes (%)"])
    stacked_bar_plotly(pct_preferencias_cat,column,"Categoria_Agrupada"
                   ,"Porcentaje_Clientes (%)",f"Preferencias por {column} por Categoria")

**Nota:** En este gráfico, y otros similares a continuación, el eje horizontal representa una dimensión de interes (ejemplo Edad) y los colores representan las categorias de comercio. Su intención es poner más enfasis en la dimensión y las diferencias entre las categorias de comercios. Es recomendado hacer doble clic en una categoria de comercio y explorarla. 

In [80]:
@interact
def porcentaje_columna_categoria(columna=columnas_preferencias):
    create_dim_percentage_stacked_chart(columna)

interactive(children=(Dropdown(description='columna', options=('Microsegmento_Edad', 'Segmento', 'Rango_Limite…

Al explorar el gráfico, se pueden apreciar las diferencias entre los segmentos de edad donde: 

- Los Jovenes prefieren el entretenimiento y comida, pero tambien la educación y tecnología.
- Los Adultos tienen preferencias variadas, pero existe mayor presencia en categorias del hogar y el dia a dia como gasolineras y supermercados.
- Los Senior prefieren seguridad y progreso en sus emprendimientos, con categorias como Instituciones Financieras, Servicios, y tambien actividades de retiro como Viajes.

Al analizarlo por segmento, se notan diferencias interesantes, algunas como:

- Los Licores son preferidos por segmentos altos, pero la vida nocturna por los segmentos bajos.
- Los Servicios comunes (telefono, agua, internet,luz), son más comunes en los segmentos bajos.
- Las compras en linea son dominadas por los Emergentes.


# Inclusión en Vista 360

Las preferencias de comercios serán incluidas en la aplicación de Vista 360. El calculo será realizado utilizando las transacciones de un cliente en el ultimo año, y utilizando el calculo del RFM. Los rangos de las variables (ejemplo la Cantidad) para los score de RFM serán guardados y leidos a la hora de crear las preferencias. 

Las preferencias serán mostradas en el perfil del cliente, en la columna preferencias. Tendrán iconos según la preferencia calculada. 

![title](http://react.rocks/images/converted/react-evil-icons.jpg)

# Conclusiones y Posibles Mejoras

Pudimos determinar las preferencias de comercios de clientes según las categorias de comercios que frecuentan. Utilizamos varias metodologias para determinarlas, y analizamos las diferencias que trae usar una sobre otra. Al final, utilizamos una que combina la recencia, frecuencia y valor monetario de las transacciones del cliente, para tener un valor más certero.

Algunas posibles mejoras: 

- Añadir datos de transacciones con tarjeta de debito. No todos usan exclusivamente su tarjeta de crédito.
- Añadir un "peso categorico" que asigna menos valor a categorias muy comunes (es decir habría menos peso de preferencia para Supermercados y Gasolineras).
- Hacer un análisis más profundo de las categorias agrupadas, añadiendo nuevas y agrupando más categorias similares.

# *FIN*